# US Quartz Import Dashboard

Mi ha ispirato il video:https://youtu.be/uhxiXOTKzfs di Thu Vu


## Impostazione strutture dati

In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator') #

import hvplot.pandas


In [2]:
df = pd.read_csv('dataweb.csv', delimiter=';',na_filter=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10260 entries, 0 to 10259
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Year     10260 non-null  int64 
 1   Unit     10260 non-null  object
 2   Country  10260 non-null  object
 3   Month    10260 non-null  int64 
 4   value    8318 non-null   object
dtypes: int64(2), object(3)
memory usage: 400.9+ KB


In [3]:
# elimina i rughe nulle
df.dropna(inplace=True)
df.head()

,Year,Unit,Country,Month,value
0,2015,usd,Romania,1,0
1,2015,usd,Pakistan,1,0
2,2015,usd,Czechia,1,64066
3,2015,usd,Germany,1,187270
4,2015,usd,Brazil,1,88613


In [4]:
# sostituisci la , con il . e converti in float
df['value'] = df['value'].str.replace(',', '.').astype(float)

In [5]:
df.head()

,Year,Unit,Country,Month,value
0,2015,usd,Romania,1,0.0
1,2015,usd,Pakistan,1,0.0
2,2015,usd,Czechia,1,64066.0
3,2015,usd,Germany,1,187270.0
4,2015,usd,Brazil,1,88613.0


In [6]:
# Elimina i valori usd/sqm e le righe con valore 0
df = df[(df['Unit']!='usd/sqm') & (df['value']!= 0)]

In [7]:
tab = pd.pivot_table(df, values=["value"], index=["Year", "Country"], columns= ["Unit"], aggfunc=np.sum)
tab

value             
Unit                       sqm          usd
Year Country                               
2015 Argentina           153.0      10597.0
     Austria              12.0       2615.0
     Belgium            2743.0     124480.0
     Brazil            15567.0    2924003.0
     Canada           513658.0   66234852.0
...                        ...          ...
2022 Thailand         481897.0   46387624.0
     Turkey           294980.0   23845984.0
     Ukraine            1821.0     127400.0
     United Arab Em     4198.0     435943.0
     Vietnam         1365536.0  134918744.0

[285 rows x 2 columns]

## Pannelli interattivi

In [8]:
idf = df.interactive()

In [9]:
# Definisci i widgets
year_slider = pn.widgets.IntSlider(name='Year slider', start=2015, end=2022, step=1, value = 2021)
year_slider 

IntSlider(end=2022, name='Year slider', start=2015, value=2021)

In [11]:
y_axis = pn.widgets.RadioButtonGroup(
    name='Unit',
    options=['sqm', 'usd',],
    button_type='success'
)
y_axis

RadioButtonGroup(button_type='success', name='Unit', options=['sqm', 'usd'], value='sqm')

In [12]:
import_pipeline = (
    idf[
        (idf['Year']==year_slider)&
        (idf['Unit']==y_axis)
    ]
    .groupby(['Country']).sum().filter(items=['Country', 'value'])
    .sort_values(by='value', ascending=False)
    .head(10)
)

/home/titotto/.virtualenvs/std/lib/python3.11/site-packages/holoviews/util/transform.py:666: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = method(*args, **kwargs)
/home/titotto/.virtualenvs/std/lib/python3.11/site-packages/holoviews/util/transform.py:666: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = method(*args, **kwargs)
/home/titotto/.virtualenvs/std/lib/python3.11/site-packages/holoviews/util/transform.py:666: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select 

In [13]:
import_pipeline

/home/titotto/.virtualenvs/std/lib/python3.11/site-packages/holoviews/util/transform.py:666: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = method(*args, **kwargs)


In [15]:
import_plot = import_pipeline.hvplot(x='Country', kind='bar', title="Major quartz exporters to USA")
import_plot

/home/titotto/.virtualenvs/std/lib/python3.11/site-packages/holoviews/util/transform.py:666: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = method(*args, **kwargs)
/home/titotto/.virtualenvs/std/lib/python3.11/site-packages/holoviews/util/transform.py:666: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = method(*args, **kwargs)


## Pubblica il pannello

In [19]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='US import statistics for quartz slabs', 
    sidebar=[pn.pane.Markdown("# Top Ten in the years"), 
             pn.pane.Markdown("#### See how in the years the portfolio of countries supplying slabs changed."),
             # pn.pane.PNG('climate_day.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),
            ],
    main=[pn.Column(import_plot.panel(width=700), y_axis,year_slider, margin=(0,25)),
         ],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();

/home/titotto/.virtualenvs/std/lib/python3.11/site-packages/holoviews/util/transform.py:666: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = method(*args, **kwargs)


** Istruzioni **

per far partire il server dalla console:

>  panel serve USImport_dashboard.ipynb